## Spark Structured Streaming - Read from Files or Directories

In [ ]:
# Create the Spark Session
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Streaming Process Files") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .master("spark://spark-master:7077") \
    .getOrCreate()

spark

In [ ]:
# To allow automatic schemaInference while reading
spark.conf.set("spark.sql.streaming.schemaInference", True)

# Create the streaming_df to read from input directory
streaming_df = spark.readStream\
    .format("json") \
    .option("cleanSource", "archive") \
    .option("sourceArchiveDir", "data/archive/device_data/") \
    .option("maxFilesPerTrigger", 1) \
    .load("data/input/")

In [ ]:
# To the schema of the data, place a sample json file and change readStream to read 
streaming_df.printSchema()
streaming_df.show(truncate=False)

In [ ]:
# Lets explode the data as devices contains list/array of device reading
from pyspark.sql.functions import explode, col

exploded_df = streaming_df \
    .select("customerId", "eventId", "eventOffset", "eventPublisher", "eventTime", "data") \
    .withColumn("devices", explode("data.devices")) \
    .drop("data")

In [ ]:
# Check the schema of the exploded_df, place a sample json file and change readStream to read 
exploded_df.printSchema()
exploded_df.show(truncate=False)

In [ ]:
# Flatten the exploded df
flattened_df = exploded_df \
    .selectExpr("customerId", "eventId", "eventOffset", "eventPublisher", "eventTime", 
                "devices.deviceId as deviceId", "devices.measure as measure", 
                "devices.status as status", "devices.temperature as temperature") 

In [ ]:
# Check the schema of the flattened_df, place a sample json file and change readStream to read 
flattened_df.printSchema()
flattened_df.show(truncate=False)

In [ ]:
# Write the output to console sink to check the output
writing_df = flattened_df.writeStream \
    .format("json") \
    .option("path", "data/output/device_data") \
    .option("checkpointLocation","checkpoint_dir") \
    .outputMode("append") \
    .start()
    
# Start the streaming application to run until the following happens
# 1. Exception in the running program
# 2. Manual Interruption
writing_df.awaitTermination()

In [ ]:
# Check the data at the output location

out_df = spark.read.json("data/output/device_data/")
out_df.show(truncate=False)